In [1]:
from selenium import webdriver
import pandas as pd
import openpyxl as op
import time
import re
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta
import numpy as np
get_number = re.compile('.*(?=\n)')
get_percent = re.compile('.*(?=%)')
get_url_without_slash = re.compile(".*[0-9a-z]")
password = !echo %password%

In [2]:
def reset():
    driver.get("https://analytics.google.com/analytics/web/?authuser=0#management/Settings/a10587421w80067081p84826225/")

def login():
    driver.get("https://analytics.google.com/analytics/web/?authuser=0#management/Settings/a10587421w80067081p84826225/")
    #아이디입력
    driver.find_element_by_css_selector('#Email').send_keys('taehun9312@gmail.com')
    driver.find_element_by_css_selector('#next').click()
    time.sleep(1)
    #비밀번호입력
    driver.find_element_by_css_selector('#Passwd').send_keys(password[0])
    driver.find_element_by_css_selector('#signIn').click()
    
def login2():
    driver.get("https://analytics.google.com/analytics/web/?authuser=0#management/Settings/a10587421w80067081p84826225/")
    #아이디입력
    driver.find_element_by_css_selector('#identifierId').send_keys('taehun9312@gmail.com')
    driver.find_element_by_css_selector('.RveJvd.snByac').click()
    time.sleep(1)
    #비밀번호입력
    driver.find_element_by_css_selector('.whsOnd.zHQkBf').send_keys(password[0])
    driver.find_element_by_css_selector('.RveJvd.snByac').click()
    
def s_to_i(value):
    value = value.replace(",","")
    value = int(value)
    return value

def check(compared_value):
    if type(compared_value) == list:
        for value in compared_value:
            if value in global_value: raise ValueError
    elif compared_value in global_value: raise ValueError
    else: print("No_duplication")
        
def get_pure_url(url):
    splited_url = url.split("/%")
    url = splited_url[0]
    url = get_url_without_slash.findall(url)[0]
    return url

def get_number_item(unprocessed_number):
    processed_number = unprocessed_number.split("(")[0]
    return s_to_i(processed_number)

        
def calculate_date(origin_date, interval, multiple_number):
    date_set = _get_date(origin_date)
    d1 = date(date_set["year"], date_set["month"], date_set["day"])
    if interval == "month":
        d2 = d1 + relativedelta(months=+1*multiple_number)
    else:
        d2 = d1 + timedelta(days = interval*multiple_number)
    next_date = _to_date(d2)
    return next_date

def _get_date(date):
    date = str(date)
    year = "20" + date[0:2]
    month = date[2:4]
    day = date[4:6]
    if month[0] == "0":
        month = month.replace("0","")
    return {"year": int(year), "month": int(month), "day": int(day)}

def _to_date(date):
    year = str(date.year)[2:4]
    if date.month < 10:
        month = "0" + str(date.month)
    else:
        month = str(date.month)
    if date.day < 10:
        day = "0" + str(date.day)
    else:    
        day = str(date.day)
    return year+month+day

In [83]:
class GA:
    
    def __init__(self, url):
        global start
        global end 
        self.df = pd.DataFrame()
        self.date = "/%3F_u.date00%3D20{start}%26_u.date01%3D20{end}".format(start= start, end=end)
        self.original_url = url
        self.url = get_pure_url(url) + self.date
        self.start = start
        self.end = end
        self.retry = 10
        self.days = 1
        self.df_start = str(start)
        self.df_end = str(end)
        self.dic_values = {}
        self.seg = "0"
        driver.get(self.url)
        
    def set_date(self, interval, number):
        self.interval = interval
        self.days = number

    def set_segment_url(self, seg, keep):
        if keep == "on": self.seg = seg
        self.url = get_url_without_slash.findall(self.url)[0] + seg
        self.df = self.df.transpose()
        #"%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ"
     #   self.go_changed_url(0)
        time.sleep(8)
        
    def set_tab(self, n_tab):
        self.tab = n_tab
        
    def set_value(self, n_value):
        if type(n_value) != list:
            self.value = [n_value]
        else:
            self.value = n_value
        
    def set_retry(self, n):
        self.retry = n
    
    def go_changed_url(self, n_try):
        start = calculate_date(self.start, self.interval, n_try)
        end = calculate_date(self.end, self.interval, n_try)
        self.df_start = str(start)
        self.df_end = str(end)
        self.date =  "/%3F_u.date00%3D20{start}%26_u.date01%3D20{end}".format(start= start, end=end)
        if len(self.seg) > 1:
            self.url = get_pure_url(self.original_url) + self.date + self.seg
        else:
            self.url = get_pure_url(self.original_url) + self.date
        driver.get(self.url)
        
    def get(self, numbers):
        for n_try in range(1, self.days+1):
            if type(numbers) == int:
                value = self._come_to_me(numbers)
                return value
            else:
                for n in numbers:
                    self.dic_values = self._fill_dic_values(n, self.dic_values)
            for title, value in self.dic_values.items():
                for n in range(0, 10):
                    try:
                        a = self.df.loc[self.df_start+" - "+self.df_end, title]
                        if str(a) == 'nan': raise KeyError
                    except KeyError:
                        if n == 0:
                            self.df.loc[self.df_start+" - "+self.df_end, title] = value
                        else:
                            title = title + str(n)
                            self.df.loc[self.df_start+" - "+self.df_end, title] = value
                        break
                    else:
                        continue
                        
                        
            if self.days > 1 and n_try < self.days: 
                    self.dic_values = {}
                    self.go_changed_url(n_try)  
        self.df = self.df.transpose()
 

    def _fill_dic_values(self, numbers, dic_values):
            n = numbers
            value = self._come_to_me(n)
            titles = self._get_title(self.value[n-1])
            
            if type(value) != list:
                if type(self.value[n-1]) != list:
                    #title 1, value 1
                    title = titles[0]
#                     if title in dic_values:
#                         title = title + str(n)
                    dic_values[title] = value
                else:
                    #titile more, value 1
                    nth = self.value[n-1][1]
                    title = titles[nth-1]
#                     if title in dic_values:
#                         title = title + str(n)
                    dic_values[title] = value
            else:
                #title more, value more
                for i, v in enumerate(value):
                    title = titles[i]
#                     if title in dic_values:
#                         title = title + str(n)
                    dic_values[title] = v 
            return dic_values
    
    def _come_to_me(self, n):
        global global_value
        error_check = "on"
        self._click_tab(n)
        value_spec = self.value[n-1]

        if type(value_spec) == list: 
            number = value_spec[0] 
            nth = value_spec[1]
        else: 
            number = value_spec
            nth = value_spec

        if type(value_spec) == list or value_spec == 1:
            value = self._get_value(nth, error_check)
            error_check = "off"
            value_for_compare = self._get_values(number, error_check)
        else:
            value_for_compare = self._get_values(number, error_check)
            value = value_for_compare
        global_value = value_for_compare
        return value

    def _get_title(self, n):
        arr_title = []
        unprocessed_titles = driver.find_elements_by_css_selector("._GArO")
        for title in unprocessed_titles:
            arr_title.append(title.text)
        return arr_title
    
    def get_percent(self, n):
        arr_value = driver.find_elements_by_css_selector("._GAj8")
        percent = arr_value[n-1].text
        processed_percent = float(get_percent.findall(percent)[0])
        print(processed_percent)
        return processed_percent
        
            
#     def get_item(self):
#         for i in range(0, self.retry):
#             try:
#                 #Screen name, Event lbael.. etc
#                 target_name = driver.find_elements_by_css_selector("._GAkub")[0].text
#                 #Unique screen view, screen view.. etc
#                 table_heads = driver.find_elements_by_css_selector("._GAYh.ACTION-sort")
#                 arr_table_heads = []
#                 for table_head in table_heads:
#                     arr_table_heads.append(table_head.text)
#                 num_cols = len(arr_table_heads)
#                 arr_table_heads.insert(0,target_name)
#                 #home_banner_1, home_banner_2.. etc
#                 target_list = driver.find_elements_by_css_selector("._GApu")
#                 #700, 600.. etc
#                 dataset = driver.find_elements_by_css_selector("._GAtjb")
#                 self.df = pd.DataFrame(columns = arr_table_heads)
#                 for row_num, target in enumerate(target_list):
#                     self.df[target_name, target] =
#                         for i in range(0, num_cols+1):
#                             df.loc[item,dataset[row_num*num_cols+i]
                        
#                         value = get_number_item(dataset[i*2+nth-1].text)
#                         check(value)
#                         print("It's ok")
#                         arr_data = []
#                         for data in dataset:
#                             arr_data.append(data.text)
#                         global_value = arr_data
#                         print(value)
#                         return value
#                 time.sleep(1)
#             except:
#                 time.sleep(4)
#                 print("It's not ok")
#                 continue
                        

    #n번째 값 가져오기
    def _get_value(self, n, error_check):
        for i in range(0, self.retry):
            try:
                print(i)
                value = driver.find_element_by_css_selector(".ID-stage{n}-top._GAhFb > ._GAXm > ._GAoCb".format(n = n)).text
                if n != 1: value = get_number.findall(value)[0]
                value = s_to_i(value)
                print("value is", value)
                if error_check == "on":
                    check(value)
                print("It's ok")
                break
            except:
                time.sleep(4)
                print("It's not ok")
                continue
        print(value)
        return value
    
    #n개의 값 가져오기
    def _get_values(self, range_n, error_check):
        arr_value = []
        for n in range(1, range_n+1):
            value = self._get_value(n, error_check)
            arr_value.append(value)
        print(arr_value)
        return arr_value

    
    #커스텀 리포트 내 탭
    def _click_tab(self, n):
        if n == 1:
            driver.get(self.url)
        else:
            for i in range(0, self.retry):
                try:
                    driver.find_elements_by_css_selector("._GAit._GAl")[n-1].click()
                    break
                except:

                    time.sleep(3)
                    print("It retry to click")
                    continue

In [4]:
# tab_category = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/C02IOMc8QBmCo6k8wf6aGQ/a10587421w80067081p102782120/%3F_u.date00%3D20170424%26_u.date01%3D20170425%26_.useg%3Dbuiltin13%26_r.tabId%3D3915/")
# tab_category.set_tab(5)
# tab_category.set_value([3,3,4,4])
# tab_category.set_date(1,30)
# tab_category.set_segment_url("%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ", "on")
# tab_category.get([1,2,3,4])

In [5]:
# excel = op.load_workbook(r"C:\Users\joseph_th\Desktop\Dashboard\Dashboard_for_GA.xlsx")
# s1 = excel["여행 Traffic overview"]["AC"]
# s2 = excel["여행 딜"]["W"]
# s3 = excel["실시간항공"]["Z"]
# s4 = excel["실시간호텔"]["U"]
# s5 = excel["APP 홈 기여"]["S"]
# excel.save(r"C:\Users\joseph_th\Desktop\Dashboard\Dashboard_for_GA_result.xlsx")

In [4]:
driver = webdriver.Chrome(executable_path=r"C:\Users\joseph_th\Downloads\chromedriver_win32\chromedriver")
login2()

In [86]:
global_value = []
start = 170603
end = 170603
reset()

In [39]:
Only_category = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/Sfz1SwoFQdmPWUdByehArw/a10587421w80067081p102782120/%3F_u.date00%3D20170518%26_u.date01%3D20170518%26_.useg%3D/")
Only_category.set_tab(2)
Only_category.set_value([[2,1],[2,1]])
Only_category.set_date(1,12)
Only_category.get([1,2])
Only_category.set_segment_url("%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ", "on")
Only_category.set_value([[2,2],[2,2]])
Only_category.get([1,2])

0
It's not ok
1
value is 603
No_duplication
It's ok
603
0
value is 603
It's ok
603
0
value is 23
It's ok
23
[603, 23]
0
value is 603
It's not ok
1
value is 4427
No_duplication
It's ok
4427
0
value is 4427
It's ok
4427
0
value is 299
It's ok
299
[4427, 299]
0
value is 4427
It's not ok
1
value is 4427
It's not ok
2
value is 2314
No_duplication
It's ok
2314
0
value is 2314
It's ok
2314
0
value is 116
It's ok
116
[2314, 116]
0
value is 2314
It's not ok
1
value is 1986
No_duplication
It's ok
1986
0
value is 1986
It's ok
1986
0
value is 105
It's ok
105
[1986, 105]
0
value is 1986
It's not ok
1
value is 3093
No_duplication
It's ok
3093
0
value is 3093
It's ok
3093
0
value is 138
It's ok
138
[3093, 138]
0
value is 3093
It's not ok
1
value is 3093
It's not ok
2
value is 1212
No_duplication
It's ok
1212
0
value is 1212
It's ok
1212
0
value is 67
It's ok
67
[1212, 67]
0
value is 1212
It's not ok
1
value is 3336
No_duplication
It's ok
3336
0
value is 3336
It's ok
3336
0
value is 185
It's ok
185
[3

In [41]:
Only_category.df.to_excel("카테고리구매.xlsx")

In [10]:
Tour_tab_purchase.df

,170601 - 170601,170602 - 170602,170603 - 170603,170604 - 170604,170605 - 170605,170606 - 170606
1. 여행홈 클릭,20283.0,17402.0,17905.0,18265.0,18502.0,21731.0
2. 아무거나 클릭,9006.0,7848.0,7659.0,8120.0,8479.0,9239.0
3. 카테고리 클릭,5594.0,4962.0,4263.0,4290.0,4435.0,4621.0
4. 기획전 배너 클릭,696.0,576.0,596.0,718.0,869.0,849.0
5. 개인화 클릭,0.0,144.0,663.0,866.0,918.0,1082.0
1. 구매완료,277.0,255.0,230.0,174.0,258.0,198.0
2. 구매완료,259.0,207.0,191.0,133.0,213.0,156.0
3. 구매완료,211.0,155.0,137.0,99.0,133.0,99.0
4. 구매완료,13.0,10.0,13.0,7.0,22.0,14.0
5. 구매완료,0.0,3.0,26.0,20.0,38.0,28.0


In [17]:
Tour_tab_purchase.df.to_excel("여행탭영역별구매1.xlsx")

In [12]:
Tour_tab_purchase = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/5a0FM1xBRpGe9ztHhLsZXQ/a10587421w80067081p102782120/%3F_u.date00%3D20170410%26_u.date01%3D20170410/")
Tour_tab_purchase.set_tab(5)
Tour_tab_purchase.set_value([1,1,1,1,1])
Tour_tab_purchase.get([1,2,3,4,5])
Tour_tab_purchase.set_segment_url("%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ", "on")
Tour_tab_purchase.retry = 5
Tour_tab_purchase.set_value([[2,2],[2,2],[2,2],[2,2],[2,2]])
Tour_tab_purchase.get([1,2,3,4,5])

0
value is 1066
It's not ok
1
value is 22156
No_duplication
It's ok
22156
0
value is 22156
It's ok
22156
[22156]
0
value is 9206
No_duplication
It's ok
9206
0
value is 9206
It's ok
9206
[9206]
0
value is 4888
No_duplication
It's ok
4888
0
value is 4888
It's ok
4888
[4888]
0
value is 812
No_duplication
It's ok
812
0
value is 812
It's ok
812
[812]
0
value is 1268
No_duplication
It's ok
1268
0
value is 1268
It's ok
1268
[1268]
0
value is 239
No_duplication
It's ok
239
0
value is 284
It's ok
284
0
value is 239
It's ok
239
[284, 239]
0
value is 245
No_duplication
It's ok
245
0
value is 271
It's ok
271
0
value is 245
It's ok
245
[271, 245]
0
value is 138
No_duplication
It's ok
138
0
value is 159
It's ok
159
0
value is 138
It's ok
138
[159, 138]
0
value is 35
No_duplication
It's ok
35
0
value is 39
It's ok
39
0
value is 35
It's ok
35
[39, 35]
0
value is 37
No_duplication
It's ok
37
0
value is 57
It's ok
57
0
value is 37
It's ok
37
[57, 37]


In [34]:
Tour_tab_purchase2.df.to_excel("여행탭 구매완료2.xlsx")

In [13]:
Tour_tab_purchase.df

,170614 - 170614
1. 여행홈 클릭,22156.0
2. 아무거나 클릭,9206.0
3. 카테고리 클릭,4888.0
4. 기획전 배너 클릭,812.0
5. 개인화 클릭,1268.0
1. 구매완료,239.0
2. 구매완료,245.0
3. 구매완료,138.0
4. 구매완료,35.0
5. 구매완료,37.0


In [16]:
Tour_tab_purchase2.df

,170614 - 170614
6. 테마 컨텐츠,395.0
7. 운영 컨텐츠,847.0
8. 할인 호텔,73.0
9. 지역/나라딜,1066.0
6. 구매완료,6.0
7. 구매완료,4.0
8. 구매완료,1.0
9. 구매완료,24.0


In [15]:
Tour_tab_purchase2 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/vEPpqfRGRsOvhhBN5-DkbQ/a10587421w80067081p102782120/%3F_u.date00%3D20170408%26_u.date01%3D20170408%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ/")
Tour_tab_purchase2.set_tab(4)
Tour_tab_purchase2.set_value([[2,1],[2,1],[2,1],[2,1]])
Tour_tab_purchase2.get([1,2,3,4])
Tour_tab_purchase2.set_segment_url("%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ", "on")
Tour_tab_purchase2.retry = 5
Tour_tab_purchase2.set_value([[2,2],[2,2],[2,2],[2,2]])
Tour_tab_purchase2.get([1,2,3,4])

0
value is 395
No_duplication
It's ok
395
0
value is 395
It's ok
395
0
value is 27
It's ok
27
[395, 27]
0
value is 847
No_duplication
It's ok
847
0
value is 847
It's ok
847
0
value is 50
It's ok
50
[847, 50]
0
value is 73
No_duplication
It's ok
73
0
value is 73
It's ok
73
0
value is 4
It's ok
4
[73, 4]
0
value is 1066
No_duplication
It's ok
1066
0
value is 1066
It's ok
1066
0
value is 81
It's ok
81
[1066, 81]
0
value is 81
It's not ok
1
value is 81
It's not ok
2
value is 6
No_duplication
It's ok
6
0
value is 6
It's ok
6
0
value is 6
It's ok
6
[6, 6]
0
value is 6
It's not ok
1
value is 6
It's not ok
2
value is 4
No_duplication
It's ok
4
0
value is 10
It's ok
10
0
value is 4
It's ok
4
[10, 4]
0
value is 4
It's not ok
1
value is 4
It's not ok
2
value is 1
No_duplication
It's ok
1
0
value is 1
It's ok
1
0
value is 1
It's ok
1
[1, 1]
0
value is 1
It's not ok
1
value is 1
It's not ok
2
value is 24
No_duplication
It's ok
24
0
value is 27
It's ok
27
0
value is 24
It's ok
24
[27, 24]


In [5]:
# html = urlopen("http://en.wikipedia.org/wiki/Kevin_Bacon")
# bsObj = BeautifulSoup(html, "html.parser")
# for link in bsObj.find("div", {"id":"bodyContent"}).findAll("a", href=re.compile("^(/wiki/)((?!:).)*$")):
#     if "href" in link.attrs:
#         print(link.attrs["href"])

In [51]:
Tab_purchase.df

,여행탭,탭내 클릭,구매완료
170601 - 170601,20283.0,228.0,225.0
170602 - 170602,17402.0,180.0,178.0
170603 - 170603,17905.0,167.0,163.0
170604 - 170604,18265.0,120.0,117.0
170605 - 170605,18502.0,185.0,183.0
170606 - 170606,21731.0,127.0,123.0
170607 - 170607,19413.0,160.0,160.0
170608 - 170608,19732.0,NaN,NaN
170609 - 170609,17622.0,NaN,NaN
170610 - 170610,19846.0,NaN,NaN


In [53]:
Tab_purchase = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/TBEAoU8fTnGAoFObTeqQdQ/a10587421w80067081p102782120/%3F_u.date00%3D20170410%26_u.date01%3D20170410%26_.useg%3DuserUILGqtAkQ6WoEA3NVyENtA/")
Tab_purchase.set_tab(4)
Tab_purchase.set_value([[2,1], 0, 1,2])
Tab_purchase.set_date(1,12)
# Tab_purchase.get([1])

Tab_purchase.set_segment_url("%26_.useg%3Duserj9MvifZ2RZ--Y5S0OHHL0w", "on")
# Tab_purchase.set_value([[2,2]])
# Tab_purchase.get([1])

# Tab_purchase.set_segment_url("%26_.useg%3DuseryUAfuE0oTlOfVJjkBz-Mcw", "on") # 4.2.2
Tab_purchase.get([1,4])


WebDriverException: Message: chrome not reachable
  (Session info: chrome=58.0.3029.110)
  (Driver info: chromedriver=2.27.440174 (e97a722caafc2d3a8b807ee115bfb307f7d2cfd9),platform=Windows NT 6.1.7601 SP1 x86_64)


In [6]:
#여행홈 이용률_And 
Tour_1 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/BNAhfOhCRYGSL64VQWI9pQ/a10587421w80067081p102782120/%3F_r.tabId%3D1169/")
Tour_1.set_tab(5)
Tour_1.set_value([2,[2,2],[2,2],[2,2],[2,2]])
#Tour_1.set_date(1,6)
Tour_1.set_segment_url("%26_.useg%3Duser3pWJ7sIwS-mDV3x54DAbeg", "on") #4.1.9v ↑
#Tour_1.set_segment_url("%26_.useg%3DuseryUAfuE0oTlOfVJjkBz-Mcw", "on") #4.2.2v ↑
Tour_1.get([1,2,3,4,5])
# Tour_1.set_segment_url("%26_.useg%3DuseryUAfuE0oTlOfVJjkBz-Mcw", "on") #4.2.2v ↑
# Tour_1.get([1,5])

Tour_2 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/hT5jydKhQni5if6X7sW0rQ/a10587421w80067081p102782120/")
Tour_2.set_tab(4)
Tour_2.set_value([[2,2],[2,2],[2,2],[2,2]])
#Tour_2.set_date(1,6)
Tour_2.set_segment_url("%26_.useg%3Duser3pWJ7sIwS-mDV3x54DAbeg", "on")
#Tour_2.set_segment_url("%26_.useg%3DuseryUAfuE0oTlOfVJjkBz-Mcw", "on") #4.2.2v ↑
Tour_2.get([1,2,3,4])

0
value is 24296
No_duplication
It's ok
24296
0
value is 4914
No_duplication
It's ok
4914
[24296, 4914]
It retry to click
It retry to click
0
value is 4914
It's not ok
1
value is 4914
It's not ok
2
value is 9152
No_duplication
It's ok
9152
0
value is 24090
It's ok
24090
0
value is 9152
It's ok
9152
[24090, 9152]
0
value is 9152
It's not ok
1
value is 2671
No_duplication
It's ok
2671
0
value is 24090
It's ok
24090
0
value is 2671
It's ok
2671
[24090, 2671]
0
value is 2671
It's not ok
1
value is 748
No_duplication
It's ok
748
0
value is 24090
It's ok
24090
0
value is 748
It's ok
748
[24090, 748]
0
value is 748
It's not ok
1
value is 1229
No_duplication
It's ok
1229
0
value is 24090
It's ok
24090
0
value is 1229
It's ok
1229
[24090, 1229]
0
value is 384
No_duplication
It's ok
384
0
value is 24296
It's ok
24296
0
value is 384
It's ok
384
[24296, 384]
It retry to click
It retry to click
0
value is 384
It's not ok
1
value is 787
No_duplication
It's ok
787
0
value is 24090
It's ok
24090
0
val

In [90]:
pd.concat([And_1.df, And_4.df, And_2_2.df]).to_excel("android.xlsx")

In [91]:
pd.concat([iOS_1.df, iOS_4.df, iOS_2_2.df]).to_excel("iOS.xlsx")

In [92]:
pd.concat([Mo_1.df, Mo_1_1.df, Mo_2_2.df]).to_excel("Moweb.xlsx")

In [93]:
pd.concat([PC_1.df, PC_1_1.df]).to_excel("PC.xlsx")

In [87]:
# #(대시보드용 1.) 카테고리 Traffic (세션)
# And_1 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/XAzFnmCtTl6SM1Cmc9OVNw/a10587421w80067081p102782120/")
# And_1.set_tab(5)
# And_1.set_value([1,1,1,1,2])
# And_1.set_date(1,15)
# And_1.get([1,2,3,4,5])

# #(대시보드용 2-2.) 항공검색(이탈율 & 국내/해외 구분)
# And_2_2 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/Xb-8k7DlTNyQjJo3M5bMjw/a10587421w80067081p102782120/%3F_u.date00%3D20170615%26_u.date01%3D20170615/")
# And_2_2.set_tab(2)
# And_2_2.set_value([2,2])
# And_2_2.set_date(1,15)
# And_2_2.get([1,2])

# #(대시보드용 4.) 트래픽 from 추천탭
# And_4 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/4rkbjB7zTGmgcRAlJaVzrg/a10587421w80067081p102782120/%3F_u.date00%3D20170615%26_u.date01%3D20170615/")
# And_4.set_tab(5)
# And_4.set_value([2,[2,2],1,2,[2,2]])
# And_4.set_date(1,15)
# And_4.get([1,2,3])

# #(대시보드용 1.) 카테고리 Traffic (세션)
# iOS_1 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/f6fCST7zQwalCMN1dpSuPw/a10587421w80067081p88823264/")
# iOS_1.set_tab(5)
# iOS_1.set_value([1,1,1,1,2])
# iOS_1.set_date(1,15)
# iOS_1.get([1,2,3,4,5])

# #(대시보드용 2-2.) 항공검색(이탈율 & 국내/해외 구분)
# iOS_2_2 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/p8shpiJuS4S2PNncs3XiDQ/a10587421w80067081p88823264/%3F_u.date00%3D20170615%26_u.date01%3D20170615/")
# iOS_2_2.set_tab(2)
# iOS_2_2.set_value([2,2])
# iOS_2_2.set_date(1,15)
# iOS_2_2.get([1,2])

# #(대시보드용 4.) 트래픽 from 추천탭
# iOS_4 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/GDfZCJv9RSC7dYyccZZ1lA/a10587421w80067081p88823264/%3F_u.date00%3D20170615%26_u.date01%3D20170615/")
# iOS_4.set_tab(5)
# iOS_4.set_value([2,[2,2],1,2,[2,2]])
# iOS_4.set_date(1,15)
# iOS_4.get([1,2,3])

# #(대시보드용 1.) 카테고리 Traffic (세션)
# Mo_1 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/V-T6iNbTQWi5USC2dSi5tA/a10587421w80067081p89357101/%3F_u.date00%3D20170615%26_u.date01%3D20170615/")
# Mo_1.set_tab(4)
# Mo_1.set_value([1,1,1,2])
# Mo_1.set_date(1,15)
# Mo_1.get([1,2,3,4])

# #(대시보드용 1-1.) 티몬 전체 세션 & 전체 카테고리 & 홈->딜
# Mo_1_1= GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/zoovm6xETN-Cd6JF9s2_hQ/a10587421w80067081p89357101/%3F_u.date00%3D20170615%26_u.date01%3D20170615/")
# Mo_1_1.set_tab(4)
# Mo_1_1.set_value([1,1,2,[2,2]])
# Mo_1_1.set_date(1,15)
# Mo_1_1.get([1,2,3,4])
#Mo_1_1.df = Mo_1_1.df[2:5]

#(대시보드용 2-2.) 항공검색(이탈율 & 국내/해외 구분)
Mo_2_2 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/ERjO4gwrTtm8N0QyMBVMSw/a10587421w80067081p89357101/%3F_u.date00%3D20170615%26_u.date01%3D20170615/")
Mo_2_2.set_tab(2)
Mo_2_2.set_value([2,2])
Mo_2_2.set_date(1,15)
Mo_2_2.get([1,2])

# #(대시보드용 1.) 카테고리 Traffic (세션)
# PC_1 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/RFQ6OlJDT5-6MMKnjtJkcA/a10587421w80067081p84826225/%3F_u.date00%3D20170615%26_u.date01%3D20170615%26_r.tabId%3D419/")
# PC_1.set_tab(2)
# PC_1.set_value([1,1])
# PC_1.set_date(1,15)
# PC_1.get([1,2])

# #(대시보드용 1-1.) 티몬 전체 세션 & 전체 카테고리 & 홈->딜
# PC_1_1 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/ZcNefO8cTRe75IKFbPcCWw/a10587421w80067081p84826225/%3F_u.date00%3D20170615%26_u.date01%3D20170615/")
# PC_1_1.set_tab(4)
# PC_1_1.set_value([1,1,2,[2,2]])
# PC_1_1.set_date(1,15)
# PC_1_1.get([1,2,3,4])

0
It's not ok
1
value is 693
No_duplication
It's ok
693
0
value is 671
No_duplication
It's ok
671
[693, 671]
0
value is 693
It's not ok
1
value is 703
No_duplication
It's ok
703
0
value is 675
No_duplication
It's ok
675
[703, 675]
0
value is 703
It's not ok
1
value is 812
No_duplication
It's ok
812
0
value is 783
No_duplication
It's ok
783
[812, 783]
0
value is 812
It's not ok
1
value is 803
No_duplication
It's ok
803
0
value is 771
No_duplication
It's ok
771
[803, 771]
0
value is 803
It's not ok
1
value is 810
No_duplication
It's ok
810
0
value is 788
No_duplication
It's ok
788
[810, 788]
0
value is 810
It's not ok
1
value is 804
No_duplication
It's ok
804
0
value is 766
No_duplication
It's ok
766
[804, 766]
0
value is 804
It's not ok
1
value is 807
No_duplication
It's ok
807
0
value is 774
No_duplication
It's ok
774
[807, 774]
0
value is 807
It's not ok
1
value is 808
No_duplication
It's ok
808
0
value is 770
No_duplication
It's ok
770
[808, 770]
0
value is 808
It's not ok
1
value is

In [23]:
#(대시보드용 1.) 카테고리 Traffic (세션)
And_1 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/XAzFnmCtTl6SM1Cmc9OVNw/a10587421w80067081p102782120/")
And_1.set_tab(5)
And_1.set_value([1,1,1,1,2])
And_1.get([1,2,3,4,5])

#(대시보드용 1-1.) 티몬 전체 세션 수 & 전체 카테고리
And_1_1 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/gmdw7utOS6OyMdXRs_jizg/a10587421w80067081p102782120/%3F_u.date00%3D20170615%26_u.date01%3D20170615/")
And_1_1.set_tab(2)
And_1_1.set_value([1,1])
And_1_1.get([1,2])

#(대시보드용 2.) 실시간 항공 메인 > 검색하기 & 여행홈 -> 항공검색결과
And_2 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/5fyYOU-QTN6tQSwRlObkUw/a10587421w80067081p102782120/%3F_u.date00%3D20170615%26_u.date01%3D20170615/")
And_2.set_tab(5)
And_2.set_value([2,2,[2,2],[2,2],2])
And_2.get([1,2,3,4,5])

#(대시보드용 2-2.) 항공검색(이탈율 & 국내/해외 구분)
And_2_2 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/Xb-8k7DlTNyQjJo3M5bMjw/a10587421w80067081p102782120/%3F_u.date00%3D20170615%26_u.date01%3D20170615/")
And_2_2.set_tab(2)
And_2_2.set_value([2,2])
And_2_2.get([1,2])

#(대시보드용 2-3.) 항공 예약하기 클릭
And_2_3 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/_yDrAZzXTaeMOGlYkwqqbw/a10587421w80067081p102782120/%3F_u.date00%3D20170615%26_u.date01%3D20170615/")
And_2_3.set_tab(2)
And_2_3.set_value([1,1])
And_2_3.get([1,2])

#(대시보드용 3.) B.com 카테고리
And_3 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/urNzazUpRDu9q-17-qtmtw/a10587421w80067081p102782120/%3F_u.date00%3D20170227%26_u.date01%3D20170305%26_.useg%3D/")
And_3.set_tab(4)
And_3.set_value([1,1,2,2])
And_3.get([1,2,3,4])

#(대시보드용 3-1.) B.com 큐레이션/이벤트
And_3_1 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/qWyiPek-Tsqgkok-rZsIjA/a10587421w80067081p102782120/%3F_u.date00%3D20170615%26_u.date01%3D20170615/")
And_3_1.set_tab(3)
And_3_1.set_value([1,2,2])
And_3_1.get([1,2,3])

#(대시보드용 4.) 트래픽 from 추천탭
And_4 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/4rkbjB7zTGmgcRAlJaVzrg/a10587421w80067081p102782120/%3F_u.date00%3D20170615%26_u.date01%3D20170615/")
And_4.set_tab(5)
And_4.set_value([2,[2,2],1,2,[2,2]])
And_4.get([1,2,3,4,5])

#(대시보드용 5.) 여행홈 트래픽
And_5 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/Yp0lRuh1QYWc4d7z640omw/a10587421w80067081p102782120/%3F_u.date00%3D20170615%26_u.date01%3D20170615/")
And_5.set_tab(4)
And_5.set_value([2,[2,2],[2,2],[2,2]])
And_5.get([1,2,3,4])

0
value is 24296
No_duplication
It's ok
24296
0
value is 9152
No_duplication
It's ok
9152
[24296, 9152]
0
value is 4914
No_duplication
It's ok
4914
0
value is 24296
It's ok
24296
0
value is 4914
It's ok
4914
[24296, 4914]
0
value is 1757
No_duplication
It's ok
1757
0
value is 24296
It's ok
24296
0
value is 1757
It's ok
1757
[24296, 1757]
0
value is 1757
It's not ok
1
value is 2671
No_duplication
It's ok
2671
0
value is 24296
It's ok
24296
0
value is 2671
It's ok
2671
[24296, 2671]


In [24]:
#(대시보드용 1.) 카테고리 Traffic (세션)
iOS_1 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/f6fCST7zQwalCMN1dpSuPw/a10587421w80067081p88823264/")
iOS_1.set_tab(5)
iOS_1.set_value([1,1,1,1,2])
iOS_1.get([1,2,3,4,5])

#(대시보드용 1-1.) 티몬 전체 세션 수 & 전체 카테고리
iOS_1_1 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/p29BV_8iRc2OovJCvhcg_Q/a10587421w80067081p88823264/%3F_u.date00%3D20170615%26_u.date01%3D20170615/")
iOS_1_1.set_tab(2)
iOS_1_1.set_value([1,1])
iOS_1_1.get([1,2])

#(대시보드용 2.) 실시간 항공 메인 > 검색하기 & 여행홈 -> 항공검색결과
iOS_2 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/2K60xdJeT2isKsPe4tgpEQ/a10587421w80067081p88823264/%3F_u.date00%3D20170615%26_u.date01%3D20170615/")
iOS_2.set_tab(5)
iOS_2.set_value([2,2,[2,2],[2,2],2])
iOS_2.get([1,2,3,4,5])

#(대시보드용 2-2.) 항공검색(이탈율 & 국내/해외 구분)
iOS_2_2 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/p8shpiJuS4S2PNncs3XiDQ/a10587421w80067081p88823264/%3F_u.date00%3D20170615%26_u.date01%3D20170615/")
iOS_2_2.set_tab(2)
iOS_2_2.set_value([2,2])
iOS_2_2.get([1,2])

#(대시보드용 2-3.) 항공 예약하기 클릭
iOS_2_3 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/btj6gUJmQ2-NZsSB1vB3tw/a10587421w80067081p88823264/%3F_u.date00%3D20170615%26_u.date01%3D20170615/")
iOS_2_3.set_tab(2)
iOS_2_3.set_value([1,1])
iOS_2_3.get([1,2])

#(대시보드용 3.) B.com 카테고리
iOS_3 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/mzJgxll5SXqATSN_oHrUiQ/a10587421w80067081p88823264/%3F_u.date00%3D20170615%26_u.date01%3D20170615/")
iOS_3.set_tab(4)
iOS_3.set_value([1,1,2,2])
iOS_3.get([1,2,3,4])

#(대시보드용 3-1.) B.com 큐레이션/이벤트
iOS_3_1 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/tZRMJC4JRhm_X9Ql6C4KOg/a10587421w80067081p88823264/%3F_u.date00%3D20170615%26_u.date01%3D20170615%26_r.tabId%3D3945/")
iOS_3_1.set_tab(3)
iOS_3_1.set_value([1,2,2])
iOS_3_1.get([1,2,3])

#(대시보드용 4.) 트래픽 from 추천탭
iOS_4 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/GDfZCJv9RSC7dYyccZZ1lA/a10587421w80067081p88823264/%3F_u.date00%3D20170615%26_u.date01%3D20170615/")
iOS_4.set_tab(5)
iOS_4.set_value([2,[2,2],1,2,[2,2]])
iOS_4.get([1,2,3,4,5])

# #(대시보드용 5.) 여행홈 트래픽
# iOS_9 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/9rdCrGeYTaCIuaCGnfLoRQ/a10587421w80067081p88823264/")
# iOS_9.set_tab(4)
# iOS_9.set_value([2,[2,2],[2,2],[2,2]])
# iOS_9.get([1,2,3,4])


0
value is 24296
It's not ok
1
value is 6313
No_duplication
It's ok
6313
0
value is 6313
It's ok
6313
[6313]
0
value is 6313
It's not ok
1
value is 3517
No_duplication
It's ok
3517
0
value is 3517
It's ok
3517
[3517]
0
value is 3517
It's not ok
1
value is 5933
No_duplication
It's ok
5933
0
value is 5933
It's ok
5933
[5933]
0
value is 5933
It's not ok
1
value is 14166
No_duplication
It's ok
14166
0
value is 14166
It's ok
14166
[14166]
0
value is 14166
It's not ok
1
value is 9637
No_duplication
It's ok
9637
0
value is 2436
No_duplication
It's ok
2436
[9637, 2436]
0
value is 9637
It's not ok
1
value is 616236
No_duplication
It's ok
616236
0
value is 616236
It's ok
616236
[616236]
0
value is 616236
It's not ok
1
value is 57569
No_duplication
It's ok
57569
0
value is 57569
It's ok
57569
[57569]
0
value is 57569
It's not ok
1
value is 5496
No_duplication
It's ok
5496
0
value is 2967
No_duplication
It's ok
2967
[5496, 2967]
0
value is 5496
It's not ok
1
value is 1189
No_duplication
It's ok
11

KeyboardInterrupt: 

In [35]:
#(대시보드용 1.) 카테고리 Traffic (세션)
Mo_1 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/V-T6iNbTQWi5USC2dSi5tA/a10587421w80067081p89357101/%3F_u.date00%3D20170615%26_u.date01%3D20170615/")
Mo_1.set_tab(4)
Mo_1.set_value([1,1,1,2])
Mo_1.get([1,2,3,4])

#(대시보드용 1-1.) 티몬 전체 세션 & 전체 카테고리 & 홈->딜
Mo_1_1= GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/zoovm6xETN-Cd6JF9s2_hQ/a10587421w80067081p89357101/%3F_u.date00%3D20170615%26_u.date01%3D20170615/")
Mo_1_1.set_tab(4)
Mo_1_1.set_value([1,1,2,[2,2]])
Mo_1_1.get([1,2,3,4])

#(대시보드용 2.) 실시간 항공 메인 > 검색하기
Mo_2 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/7aMTsZECQu2GVnxwp3IXRg/a10587421w80067081p89357101/%3F_u.date00%3D20170615%26_u.date01%3D20170615/")
Mo_2.set_tab(4)
Mo_2.set_value([2,2,[2,2],[2,2]])
Mo_2.get([1,2,3,4])

#(대시보드용 2-2.) 항공검색(이탈율 & 국내/해외 구분)
Mo_2_2 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/ERjO4gwrTtm8N0QyMBVMSw/a10587421w80067081p89357101/%3F_u.date00%3D20170615%26_u.date01%3D20170615/")
Mo_2_2.set_tab(2)
Mo_2_2.set_value([2,2])
Mo_2_2.get([1,2])
                  
#(대시보드용 2-3.) 항공 예약하기 클릭
Mo_2_3 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/Shs74NpmR1Wcy5isHt_ZGA/a10587421w80067081p89357101/%3F_u.date00%3D20170615%26_u.date01%3D20170615/")
Mo_2_3.set_tab(2)
Mo_2_3.set_value([1,1])
Mo_2_3.get([1,2])
                  
#(대시보드용 3.) B.com 카테고리/큐레이션/이벤트
Mo_3 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/W52iAReKSeydMYwmwkpBbA/a10587421w80067081p89357101/%3F_u.date00%3D20170615%26_u.date01%3D20170615/")
Mo_3.set_tab(3)
Mo_3.set_value([2,2,2])
Mo_3.get([1,2,3])

0
value is 18
It's not ok
1
value is 658
No_duplication
It's ok
658
0
value is 626
No_duplication
It's ok
626
[658, 626]
0
value is 658
It's not ok
1
value is 739
No_duplication
It's ok
739
0
value is 719
No_duplication
It's ok
719
[739, 719]
0
value is 53
No_duplication
It's ok
53
0
value is 53
It's ok
53
[53]
0
value is 53
It's not ok
1
value is 117
No_duplication
It's ok
117
0
value is 117
It's ok
117
[117]
0
value is 145
No_duplication
It's ok
145
0
value is 87
No_duplication
It's ok
87
[145, 87]
0
value is 238
No_duplication
It's ok
238
0
value is 39
No_duplication
It's ok
39
[238, 39]
0
value is 18
No_duplication
It's ok
18
0
value is 2
No_duplication
It's ok
2
[18, 2]


In [27]:
#(대시보드용 1.) 카테고리 Traffic (세션)
PC_1 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/RFQ6OlJDT5-6MMKnjtJkcA/a10587421w80067081p84826225/%3F_u.date00%3D20170615%26_u.date01%3D20170615%26_r.tabId%3D419/")
PC_1.set_tab(2)
PC_1.set_value([1,1])
PC_1.get([1,2])

#(대시보드용 1-1.) 티몬 전체 세션 & 전체 카테고리 & 홈->딜
PC_1_1 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/ZcNefO8cTRe75IKFbPcCWw/a10587421w80067081p84826225/%3F_u.date00%3D20170615%26_u.date01%3D20170615/")
PC_1_1.set_tab(4)
PC_1_1.set_value([1,1,2,[2,2]])
PC_1_1.get([1,2,3,4])

#(대시보드용 2.) 실시간 항공 메인 > 검색하기
PC_2 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/KkWuVj5-SCOH7ddtrf4bOA/a10587421w80067081p84826225/%3F_u.date00%3D20170615%26_u.date01%3D20170615/")
PC_2.set_tab(2)
PC_2.set_value([2,2])
PC_2.get([1,2])

#(대시보드용 2-2.) 항공 예약하기 클릭
PC_2_2 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/YspUfeZLQ7KyhFzRxehRcQ/a10587421w80067081p84826225/%3F_u.date00%3D20170615%26_u.date01%3D20170615/")
PC_2_2.set_tab(2)
PC_2_2.set_value([1,1])
PC_2_2.get([1,2])

#(대시보드용 3.) B.com 카테고리/큐레이션/이벤트
PC_3 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/jQo-QVnGRrG1Y_Dtaoz0FQ/a10587421w80067081p84826225/%3F_u.date00%3D20170615%26_u.date01%3D20170615/")
PC_3.set_tab(4)
PC_3.set_value([1,2,2,2])
PC_3.get([1,2,3,4])

0
value is 8976
It's not ok
1
value is 13271
No_duplication
It's ok
13271
0
value is 13271
It's ok
13271
[13271]
0
value is 13271
It's not ok
1
value is 6416
No_duplication
It's ok
6416
0
value is 6416
It's ok
6416
[6416]
0
value is 6416
It's not ok
1
value is 567625
No_duplication
It's ok
567625
0
value is 567625
It's ok
567625
[567625]
0
value is 567625
It's not ok
1
value is 64252
No_duplication
It's ok
64252
0
value is 64252
It's ok
64252
[64252]
0
value is 64252
It's not ok
1
value is 228292
No_duplication
It's ok
228292
0
value is 384
No_duplication
It's ok
384
[228292, 384]
0
value is 384
It's not ok
1
value is 999
No_duplication
It's ok
999
0
value is 228292
It's ok
228292
0
value is 999
It's ok
999
[228292, 999]
0
value is 228292
It's not ok
1
value is 2504
No_duplication
It's ok
2504
0
value is 1627
No_duplication
It's ok
1627
[2504, 1627]
0
value is 2504
It's not ok
1
value is 2198
No_duplication
It's ok
2198
0
value is 1300
No_duplication
It's ok
1300
[2198, 1300]
0
value i

In [18]:
FIT_search_ad = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/Xb-8k7DlTNyQjJo3M5bMjw/a10587421w80067081p102782120/%3F_u.date00%3D20170501%26_u.date01%3D20170507/")
FIT_search_ad.set_tab(2)
FIT_search_ad.set_value([2,2])
FIT_search_ad.get([1,2])
FIT_search_iOS = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/p8shpiJuS4S2PNncs3XiDQ/a10587421w80067081p88823264/")
FIT_search_iOS.set_tab(2)
FIT_search_iOS.set_value([2,2])
FIT_search_iOS.get([1,2])
FIT_search_Moweb = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/ERjO4gwrTtm8N0QyMBVMSw/a10587421w80067081p89357101/")
FIT_search_Moweb.set_tab(2)
FIT_search_Moweb.set_value([2,2])
FIT_search_Moweb.get([1,2])

0
value is 1310792
It's not ok
1
value is 21587
No_duplication
It's ok
21587
0
value is 20225
No_duplication
It's ok
20225
[21587, 20225]
0
value is 21587
It's not ok
1
value is 21587
It's not ok
2
value is 14847
No_duplication
It's ok
14847
0
value is 14346
No_duplication
It's ok
14346
[14847, 14346]
0
value is 14847
It's not ok
1
value is 11040
No_duplication
It's ok
11040
0
value is 10468
No_duplication
It's ok
10468
[11040, 10468]
0
value is 11040
It's not ok
1
value is 10005
No_duplication
It's ok
10005
0
value is 9577
No_duplication
It's ok
9577
[10005, 9577]
0
value is 10005
It's not ok
1
value is 5477
No_duplication
It's ok
5477
0
value is 5105
No_duplication
It's ok
5105
[5477, 5105]
0
value is 5477
It's not ok
1
value is 4796
No_duplication
It's ok
4796
0
value is 4566
No_duplication
It's ok
4566
[4796, 4566]


In [77]:
pd.concat([And_6.df, And_7.df, iOS_6.df,iOS_7.df,Mo_6.df,PC_6.df])

,170522 - 170528
And 시스템 큐레이션 2. 주말특가 view,6743.0
And 시스템 큐레이션 2. 주말특가 클릭,580.0
And 카테고리 1. 여행홈 -> B.com(실시간 호텔),2218.0
And 카테고리 2. 국내호텔 배너클릭,20.0
And 카테고리 3. 해외호텔 메인 검색모듈 view,4129.0
And 카테고리 3. 해외호텔 메인 검색모듈 -> 이동,2966.0
And 시스템 큐레이션 1. 여행홈 -> B.com,835.0
And_ 매뉴얼큐레이션 - 1.큐레이션 딜view,3463.0
매뉴얼큐레이션 - 1.큐레이션 딜 view -> 이동,997.0
And_이벤트 페이지,8925.0


In [20]:
# #국내, 해외, 항공호텔, 전체카테고리
df1 = pd.concat([And_1.df, iOS_1.df, Mo_1.df, PC_1.df])

# #티몬 전체 카테고리
df2 = pd.concat([And_2.df, iOS_2.df, Mo_2.df, PC_2.df])

# #국내딜, 해외딜, 전체딜
#df3 = pd.concat([And_3.df, iOS_3.df, Mo_3.df, PC_3.df])

# #항공검색 해외, 국내, 해외 -> 호텔메인, 해외 -> 국내메인
df4 = pd.concat([And_4.df, iOS_4.df, Mo_4.df, PC_4.df])

# #항공예약클릭 국내, 해외
df5 = pd.concat([And_5.df, iOS_5.df, Mo_5.df, PC_5.df])

# #부킹닷컴
df6 = pd.concat([And_6.df, And_7.df, iOS_6.df,iOS_7.df,Mo_6.df,PC_6.df])

# #홈 -> 딜상세(국내), 해외, 여행카테고리숏컷
df7 = pd.concat([And_8.df, iOS_8.df, Mo_7.df, PC_7.df])

# #홈 -> 콜렉션(국내), 해외
df8 = pd.concat([And_9.df, iOS_9.df])
# #여행홈 -> 카테고리, 항공검결
df9 = pd.concat([And_10.df, iOS_10.df])

# # 실시간 항공 국내/해외 구분
df10 = pd.concat([FIT_search_ad.df, FIT_search_iOS.df, FIT_search_Moweb.df])

In [35]:
df6

,170529 - 170604
And 시스템 큐레이션 2. 주말특가 view,7791.0
And 시스템 큐레이션 2. 주말특가 클릭,639.0
And 카테고리 1. 여행홈 -> B.com(실시간 호텔),2057.0
And 카테고리 2. 국내호텔 배너클릭,26.0
And 카테고리 3. 해외호텔 메인 검색모듈 view,4440.0
And 카테고리 3. 해외호텔 메인 검색모듈 -> 이동,3192.0
And 시스템 큐레이션 1. 여행홈 -> B.com,625.0
And_ 매뉴얼큐레이션 - 1.큐레이션 딜view,2658.0
매뉴얼큐레이션 - 1.큐레이션 딜 view -> 이동,756.0
And_이벤트 페이지,5586.0


In [36]:
df7

,170529 - 170604
And_홈화면,7555101.0
And_딜상세(국내),39833.0
And_딜상세(해외),8274.0
And_여행카테고리 숏컷,2163.0
iOS_홈화면,2571224.0
iOS_딜상세(국내),2859.0
iOS_딜상세(해외),1598.0
iOS_카테고리숏컷,13670.0
Moweb_홈화면,755056.0
Moweb_딜상세(국내),4524.0


In [24]:
# #B.com 카테고리/시스템 큐레이션
# And_6 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/urNzazUpRDu9q-17-qtmtw/a10587421w80067081p102782120/%3F_u.date00%3D20170501%26_u.date01%3D20170507/")
# And_6.set_tab(5)
# And_6.set_value([2,1,1,2,1])
# And_6.set_date(7,3)
# And_6.get([1,2,3,4,5])

# #B.com 매뉴얼 큐레이션/이벤트
# And_7 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/qWyiPek-Tsqgkok-rZsIjA/a10587421w80067081p102782120/%3F_u.date00%3D20170501%26_u.date01%3D20170507%26_r.tabId%3D210/")
# And_7.set_tab(2)
# And_7.set_value([2,2])
# And_7.set_date(7,3)
# And_7.get([1,2])

# #B.com 카테고리/시스템 큐레이션
# iOS_6 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/mzJgxll5SXqATSN_oHrUiQ/a10587421w80067081p88823264/")
# iOS_6.set_tab(5)
# iOS_6.set_value([2,1,1,2,1])
# iOS_6.set_date(7,3)
# iOS_6.get([1,2,3,4,5])

# #B.com 매뉴얼 큐레이션/이벤트
# iOS_7 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/tZRMJC4JRhm_X9Ql6C4KOg/a10587421w80067081p88823264/")
# iOS_7.set_tab(2)
# iOS_7.set_value([2,2])
# iOS_7.set_date(7,3)
# iOS_7.get([1,2])

# #부킹닷컴 큐레이션, 호텔메인, 이벤트
# Mo_6 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/W52iAReKSeydMYwmwkpBbA/a10587421w80067081p89357101/%3F_r.tabId%3D2101/")
# Mo_6.set_tab(3)
# Mo_6.set_value([2,2,2])
# Mo_6.set_date(7,3)
# Mo_6.get([1,2,3])

# # B.com 카테고리/시스템/매뉴얼큐레이션/이벤트
# PC_6 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/jQo-QVnGRrG1Y_Dtaoz0FQ/a10587421w80067081p84826225/%3F_u.date00%3D20170501%26_u.date01%3D20170507%26_r.tabId%3D4292/")
# PC_6.set_tab(3)
# PC_6.set_value([1,2,2,1])
# PC_6.set_date(7,3)
# PC_6.get([1,2,3,4])

In [21]:
# #######월간 여행딜

# #국내, 해외, 여행전체 딜
# And_3 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/c9agY1srRx2PaD8JAkK0yw/a10587421w80067081p102782120/")
# And_3.set_tab(3)
# And_3.set_value([3,3,3])
# And_3.get([1,2,3])

# And_3.set_segment_url("%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ", "off")
# And_3.set_value([[3,3],[3,3],[3,3]])
# And_3.get([1,2,3])

# #국내딜, 해외딜, 전체딜
# iOS_3 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/A9m3wyZgQTmxJrO3lQ6UMw/a10587421w80067081p88823264/%3F_u.date00%3D20170327%26_u.date01%3D20170402%26_r.tabId%3D4646%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ/")
# iOS_3.set_tab(3)
# iOS_3.set_value([3,3,3])
# iOS_3.get([1,2,3])

# iOS_3.set_segment_url("%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ", "off")
# iOS_3.set_value([[3,3],[3,3],[3,3]])
# iOS_3.get([1,2,3])

# #여행딜 전체, 국내, 해외
# Mo_3 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/upJUz96ERVGUIE2WWd8Tgg/a10587421w80067081p89357101/")
# Mo_3.set_tab(3)
# Mo_3.set_value([3,3,3])
# Mo_3.get([1,2,3])

# Mo_3.set_segment_url("%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ", "off")
# Mo_3.set_value([[3,3],[3,3],[3,3]])
# Mo_3.get([1,2,3])

# # 여행 전체, 국내, 해외딜
# PC_3 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/hRoEL4w4RCyYpcmhDbvgWw/a10587421w80067081p84826225/%3F_.useg%3D/")
# PC_3.set_tab(3)
# PC_3.set_value([3,3,3])
# PC_3.get([1,2,3])

# PC_3.set_segment_url("%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ", "off")
# PC_3.set_value([[3,3],[3,3],[3,3]])
# PC_3.get([1,2,3])

In [ ]:
#부킹닷컴 큐레이션, 호텔배너, 주말특가, 검색모듈
And_6 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/urNzazUpRDu9q-17-qtmtw/a10587421w80067081p102782120/%3F_u.date00%3D20170227%26_u.date01%3D20170305%26_.useg%3D/")
And_6.set_tab(4)
And_6.set_value([2,1,2,2])
And_6.get([1,2,3,4])
#부킹닷컴 큐레이션, 호텔배너, 주말특가, 해외호텔검색모듈
iOS_6 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/fiA5QbsjQAeJxkmaYUFJEQ/a10587421w80067081p88823264/")
iOS_6.set_tab(4)
iOS_6.set_value([2,1,2,2])
iOS_6.get([1,2,3,4])
#부킹닷컴 큐레이션, 호텔메인
Mo_6 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/W52iAReKSeydMYwmwkpBbA/a10587421w80067081p89357101/")
Mo_6.set_tab(2)
Mo_6.set_value([2,2])
Mo_6.get([1,2])
# 부킹닷컴 큐레이션, 해외호텔 카테고리, 주말특가
PC_6 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/jQo-QVnGRrG1Y_Dtaoz0FQ/a10587421w80067081p84826225/%3F_.useg%3D/")
PC_6.set_tab(3)
PC_6.set_value([2,1,2])
PC_6.get([1,2,3])

In [14]:
# booking_df_User = pd.concat([And_6.df, iOS_6.df, Mo_6.df, PC_6.df])

In [49]:
# #######월간항공

# #항공검색 해외, 국내, 해외->호텔메인, 해외->국내항공메인
# And_4 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/5fyYOU-QTN6tQSwRlObkUw/a10587421w80067081p102782120/%3F_u.date00%3D20170101%26_u.date01%3D20170301%26_.useg%3Duser-3-OozmyTj6PoaE9MqRGzQ/")
# And_4.set_tab(4)
# And_4.set_value([2,2,[2,2],[2,2]])
# And_4.get([1,2,3,4])

# #항공예약 클릭 국내, 해외 
# And_5 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/_yDrAZzXTaeMOGlYkwqqbw/a10587421w80067081p102782120/%3F_u.date00%3D20170227%26_u.date01%3D20170305%26_.useg%3D%26_r.tabId%3D6028/")
# And_5.set_tab(2)
# And_5.set_value([1,1])
# And_5.get([1,2])

# #항공검색 해외, 국내, 해외 -> 호텔메인, 해외 -> 국내메인
# iOS_4 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/2K60xdJeT2isKsPe4tgpEQ/a10587421w80067081p88823264/%3F_u.date00%3D20170301%26_u.date01%3D20170320/")
# iOS_4.set_tab(4)
# iOS_4.set_value([2,2,[2,2],[2,2]])
# iOS_4.get([1,2,3,4])

# #항공예약클릭 국내, 해외
# iOS_5 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/btj6gUJmQ2-NZsSB1vB3tw/a10587421w80067081p88823264/%3F_u.date00%3D20170301%26_u.date01%3D20170331%26_.useg%3D/")
# iOS_5.set_tab(2)
# iOS_5.set_value([1,1])
# iOS_5.get([1,2])

# #항공검색 해외, 국내, 해외->호텔, 해외->국내
# Mo_4 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/7aMTsZECQu2GVnxwp3IXRg/a10587421w80067081p89357101/")
# Mo_4.set_tab(4)
# Mo_4.set_value([2,2,[2,2],[2,2]])
# Mo_4.get([1,2,3,4])

# #항공예약클릭 국내, 해외
# Mo_5 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/Shs74NpmR1Wcy5isHt_ZGA/a10587421w80067081p89357101/")
# Mo_5.set_tab(2)
# Mo_5.set_value([1,1])
# Mo_5.get([1,2])

# # 항공검색 국내, 해외
# PC_4 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/KkWuVj5-SCOH7ddtrf4bOA/a10587421w80067081p84826225/")
# PC_4.set_tab(2)
# PC_4.set_value([2,2])
# PC_4.get([1,2])

# # 예약클릭 국내, 해외
# PC_5 = GA("https://analytics.google.com/analytics/web/?authuser=0#my-reports/YspUfeZLQ7KyhFzRxehRcQ/a10587421w80067081p84826225/%3F_.useg%3D/")
# PC_5.set_tab(2)
# PC_5.set_value([2,2])
# PC_5.get([1,2])

In [48]:
# pd.concat([And_4.df, And_5.df, iOS_4.df, iOS_5.df, Mo_4.df, Mo_5.df, PC_4.df, PC_5.df])